In [1]:
import datetime
import algorithms
import data_clean
import simulations
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
df = data_clean.do(data_clean)
df = algorithms.ml_probabilities(df)
df_historic_win_data = data_clean.clean_data("spreadspoke_scores.csv", "nfl_teams.csv", "1990-9-10") #winners by spread since
#win_dict = algorithms.win_percent(df_historic_win_data)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [3]:
# start_date, end_date = simulations.historic_win_set(2018,11,15, 2017, 9, 10)
# my_sim = simulations.Simulations(1000,0.05, -3, start_date, end_date) #spread is negative
# my_sim.simulate_v2(df_historic_win_data,df,algorithms)
# 
# 

In [4]:
# x = my_sim.current_amount

In [5]:
start_date, end_date = simulations.historic_win_set(2020,11,15,2009,9,10)
min_ev = [.05]
max_spread = [-15]
df_results = pd.DataFrame()
for ev in min_ev:
    for spread in max_spread:
        my_sim = simulations.Simulations(1000,ev, spread, start_date, end_date) #spread is negative
        my_sim.simulate_v2(df_historic_win_data,df,algorithms)
        results = {'Amount' : round(my_sim.current_amount,2),'Amount_EV' : round(my_sim.current_ev,2), 'Max_Spread': spread,
                                                     'EV' : ev, 'Bet_Count': my_sim.bet}
        df_results = df_results.append(results, ignore_index=True)


        my_file = "test_10.csv"
        df_results.to_csv(my_file, header=True, index=False)

In [8]:
df_results = df_results.sort_values(by="Amount", ascending=False)
df_results

KeyError: 'Amount'

In [2]:

import datetime
import algorithms
import data_clean
import simulations
import pandas as pd
import numpy as np
from pathlib import Path

df = data_clean.clean_data("spreadspoke_scores.csv", "nfl_teams.csv", "2009-9-10")
df = data_clean.make_simple(df)

df_teams = pd.read_csv("nfl_teams.csv")

In [3]:
scores = "NFL_spread_since2009.csv"
df_spread = pd.read_csv(scores)

print(df_spread.shape)

(6862, 19)


In [4]:
def remove_misc_teams(bad_data, df_ml):  # used on ml df. Removes random teams from dataset
    df_ml = df_ml[~df_ml['team'].isin(bad_data)]
    df_ml = df_ml[~df_ml['opp_team'].isin(bad_data)]
    return df_ml

In [5]:
bad_data = ['NFC', 'AFC', 'Team Rice', 'Team Sanders', 'Team Carter', 'Team Irvin']
df_spread = remove_misc_teams(bad_data, df_spread)


In [7]:
print(df_spread.shape)
df_spread.head()


(6856, 19)


,key,date,H/A,team,opp_team,pinnacle_line,pinnacle_odds,5dimes_line,5dimes_odds,heritage_line,heritage_odds,bovada_line,bovada_odds,betonline_line,betonline_odds,bet365_line,bet365_odds,bodog_line,bodog_odds
0,20090913_Miami_Atlanta,20090913,away,Miami,Atlanta,+4,-104,+4,-104,NaN,NaN,+4,-105,+4,-110,NaN,NaN,+4,-110
1,20090913_Miami_Atlanta,20090913,home,Atlanta,Miami,-4,-104,-4,-106,NaN,NaN,-4,-115,-4,-110,NaN,NaN,-4,-110
2,20090913_Kansas City_Baltimore,20090913,away,Kansas City,Baltimore,+13,-112,+12.5,-105,NaN,NaN,+13,-105,+13,-110,NaN,NaN,+13,-110
3,20090913_Kansas City_Baltimore,20090913,home,Baltimore,Kansas City,-13,+104,-12.5,-105,NaN,NaN,-13,-115,-13,-110,NaN,NaN,-13,-110
4,20090913_Philadelphia_Carolina,20090913,away,Philadelphia,Carolina,-1,-120,-2.5,-109,NaN,NaN,-3,+100,-2.5,-110,NaN,NaN,-2.5,-110


In [5]:
def add_spread_team_id(self, df_ml, df_teams,
                   csvfile="team_key_value_pairs.csv"):  # changes team names to team id by merged df_ml, df_team, and a key value pair table
    df_kv = pd.read_csv(csvfile)
    df_ml = df_ml.merge(df_kv, left_on='team', right_on='team', how='left')
    df_ml = df_ml.merge(df_teams[['team_name', 'team_id']], left_on='team_name', right_on='team_name', how='left')
    df_ml = df_ml.drop("team_name", axis=1)
    df_ml = df_ml.merge(df_kv, left_on='opp_team', right_on='team', how='left')
    df_ml = df_ml.merge(df_teams[['team_name', 'team_id']], left_on='team_name', right_on='team_name', how='inner')
    df_ml = df_ml.drop(["team_x", "opp_team", "team_name", "team_y", "team_name"], axis=1)
    df_ml = df_ml.rename(columns={"team_id_x": "home", "team_id_y": "away"})

    df_ml['spread'] = df_ml[['pinnacle_line', '5dimes_line', 'heritage_line	', 'betonline_line', 'bet365_line', 'bodog_line']].values.tolist()
    df_ml = df_ml.drop(['pinnacle_line', '5dimes_line', 'heritage_line	', 'betonline_line', 'bet365_line', 'bodog_line'], axis=1)
    df_ml['spread'] = df_ml['spread'].apply(lambda x: [i for i in x if i == i])
    df_ml['spread'] = df_ml['spread'].apply(lambda x: self.median_split_special_case(x))
    # use median of the lines for the moneyline (will try out other methods in future). This is mostly for backtesting.
    df_ml_home = df_ml[
        df_ml['H/A'] == 'home']  # I would do if postive, highest ML, if negative, then number closest to 0.
    df_ml_away = df_ml[df_ml['H/A'] == 'away']
    df_ml = df_ml_home.merge(df_ml_away[['key', 'moneyline']], on="key")
    df_ml = df_ml.rename(columns={"moneyline_x": "home_spread", "moneyline_y": "away_spread"})
    df_ml['date'] = pd.to_datetime(df_ml['date'], format='%Y%m%d')  # date
    return df_ml


In [48]:
import datetime
import algorithms
import data_clean
import simulations
import pandas as pd
import numpy as np
from pathlib import Path

df = data_clean.clean_data("spreadspoke_scores.csv", "nfl_teams.csv", "2009-9-10")
df = data_clean.make_simple(df)
df

,schedule_date,home,score_home,away,score_away,team_favorite_id,spread_favorite
0,2009-09-13,ARI,16.0,SF,20.0,ARI,-4.5
1,2009-09-13,ATL,19.0,MIA,7.0,ATL,-4.0
2,2009-09-13,BAL,38.0,KC,24.0,BAL,-13.0
3,2009-09-13,CAR,10.0,PHI,38.0,PHI,-2.5
4,2009-09-13,CIN,7.0,DEN,12.0,CIN,-5.0
...,...,...,...,...,...,...,...
3050,2020-11-01,KC,35.0,NYJ,9.0,KC,-19.5
3051,2020-11-01,MIA,28.0,LAR,17.0,LAR,-3.5
3052,2020-11-01,PHI,23.0,DAL,9.0,PHI,-11.5
3053,2020-11-01,SEA,37.0,SF,27.0,SEA,-3.0


In [66]:
df_spread = pd.read_csv("NFL_spread_since2009.csv")
bad_data = ['NFC', 'AFC', 'Team Rice', 'Team Sanders', 'Team Carter', 'Team Irvin']
df_spread = data_clean.remove_misc_teams(bad_data, df_spread)
df_ml = pd.read_csv("NFL_moneylines.csv")
df_ml = data_clean.remove_misc_teams(bad_data, df_ml)
df_ml = data_clean.add_ml_team_id(data_clean, df_ml, df_teams)
#df_spread = data_clean.add_spread_team_id(data_clean, df_spread, df_teams)
# df_test1 = data_clean.update_spreads(df, df_spread)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [67]:
def add_spread_team_id(df_ml, df_teams,
                       csvfile="team_key_value_pairs.csv"):  # changes team names to team id by merged df_ml, df_team, and a key value pair table
    df_kv = pd.read_csv(csvfile)
    df_ml = df_ml.merge(df_kv, left_on='team', right_on='team', how='left')
    df_ml = df_ml.merge(df_teams[['team_name', 'team_id']], left_on='team_name', right_on='team_name', how='left')
    df_ml = df_ml.drop("team_name", axis=1)
    df_ml = df_ml.merge(df_kv, left_on='opp_team', right_on='team', how='left')
    df_ml = df_ml.merge(df_teams[['team_name', 'team_id']], left_on='team_name', right_on='team_name', how='inner')
    df_ml = df_ml.drop(["team_x", "opp_team", "team_name", "team_y", "team_name"], axis=1)
    df_ml = df_ml.rename(columns={"team_id_x": "home", "team_id_y": "away"})

    df_ml['spread'] = df_ml[['pinnacle_line', '5dimes_line', 'heritage_line', 'betonline_line', 'bet365_line',
                             'bodog_line']].values.tolist()
    df_ml = df_ml.drop(['pinnacle_line', '5dimes_line', 'heritage_line', 'betonline_line', 'bet365_line', 'bodog_line'],
                       axis=1)
    df_ml['spread'] = df_ml['spread'].apply(lambda x: [float(i) for i in x if is_float(i)])
    df_ml['spread'] = df_ml['spread'].apply(lambda x: [i for i in x if i == i])
    df_ml['spread'] = df_ml['spread'].apply(lambda x: np.median(x))
    df_ml_home = df_ml[
    df_ml['H/A'] == 'home']  # I would do if postive, highest ML, if negative, then number closest to 0.
    df_ml_away = df_ml[df_ml['H/A'] == 'away']
    
    df_ml = df_ml_home.merge(df_ml_away[['key', 'spread']], on="key")
    df_ml = df_ml.rename(columns={"spread_x": "home_spread", "spread_y": "away_spread"})
    df_ml['date'] = pd.to_datetime(df_ml['date'], format='%Y%m%d')
    
    df_ml['spread_favorite'] = np.where(df_ml['home_spread'] >= df_ml['away_spread'], df_ml['away_spread'],
                                        df_ml['home_spread'])
    df_ml['team_favorite_id'] = np.where(df_ml['home_spread'] >= df_ml['away_spread'], df_ml['away'], df_ml['home'])
    df_ml = df_ml[['date', 'home', 'away', 'team_favorite_id', 'spread_favorite']]
    return df_ml

In [68]:
def is_float(n):  # checks to see if number is a real number else returns false
    try:
        float(n)
        return True
    except ValueError:
        return False

In [69]:
df_teams = pd.read_csv("nfl_teams.csv")

df_ml = add_spread_team_id(df_spread, df_teams)
df_ml.head()

,date,home,away,team_favorite_id,spread_favorite
0,2009-09-27,NE,ATL,NE,-4.50
1,2009-10-11,SF,ATL,SF,-1.00
2,2009-11-02,NO,ATL,NO,-11.00
3,2009-11-15,CAR,ATL,ATL,-1.00
4,2009-11-22,NYG,ATL,NYG,-7.25


In [53]:
df_ml_home = df_ml[
    df_ml['H/A'] == 'home']  # I would do if postive, highest ML, if negative, then number closest to 0.
df_ml_away = df_ml[df_ml['H/A'] == 'away']

df_ml = df_ml_home.merge(df_ml_away[['key', 'spread']], on="key")
df_ml = df_ml.rename(columns={"spread_x": "home_spread", "spread_y": "away_spread"})
df_ml['date'] = pd.to_datetime(df_ml['date'], format='%Y%m%d')

df_ml['spread_favorite'] = np.where(df_ml['home_spread'] >= df_ml['away_spread'], df_ml['away_spread'],
                                    df_ml['home_spread'])
df_ml['team_favorite_id'] = np.where(df_ml['home_spread'] >= df_ml['away_spread'], df_ml['away'], df_ml['home'])
df_ml = df_ml[['date', 'home', 'away', 'team_favorite_id', 'spread_favorite']]

,key,date,H/A,pinnacle_odds,5dimes_odds,heritage_odds,bovada_line,bovada_odds,betonline_odds,bet365_odds,bodog_odds,home,away,home_spread,away_spread,spread_favorite,favorite_team_id
0,20090927_Atlanta_New England,2009-09-27,home,+101,-102,NaN,-4.5,-110,-110,NaN,-110,NE,ATL,-4.50,4.50,-4.50,NE
1,20091011_Atlanta_San Francisco,2009-10-11,home,+105,+100,NaN,-1,-110,-110,NaN,PK-110,SF,ATL,-1.00,1.00,-1.00,SF
2,20091102_Atlanta_New Orleans,2009-11-02,home,+101,+110,NaN,-11.5,-110,-110,NaN,-110,NO,ATL,-11.00,11.00,-11.00,NO
3,20091115_Atlanta_Carolina,2009-11-15,home,-108,PK-105,NaN,+2,-110,-110,NaN,-110,CAR,ATL,1.00,-1.00,-1.00,ATL
4,20091122_Atlanta_N.Y. Giants,2009-11-22,home,-110,+105,NaN,-7,-115,-110,NaN,-115,NYG,ATL,-7.25,7.25,-7.25,NYG
5,20091220_Atlanta_N.Y. Jets,2009-12-20,home,+103,-110,NaN,-5,-110,-110,NaN,-110,NYJ,ATL,-5.00,5.00,-5.00,NYJ
6,20100103_Atlanta_Tampa Bay,2010-01-03,home,-108,+105,NaN,+2,+120,+100,NaN,-120,TB,ATL,2.75,-2.75,-2.75,ATL
7,20100827_Atlanta_Miami,2010-08-27,home,-115,-115,NaN,-2.5,-110,-115,NaN,-120,MIA,ATL,-2.50,2.50,-2.50,MIA
8,20100902_Atlanta_Jacksonville,2010-09-02,home,+101,+100,NaN,-3,-105,-105,NaN,-105,JAX,ATL,-3.00,3.00,-3.00,JAX
9,20100912_Atlanta_Pittsburgh,2010-09-12,home,-102,-102,NaN,+2,-110,-110,NaN,-110,PIT,ATL,1.00,-1.00,-1.00,ATL


In [74]:
def update_spreads(df, df_spread_update): #updated spread since 2009. Still need a way to get score
    df_spread_update = df.merge(df_spread_update,left_on=['schedule_date', 'home', 'away'], right_on=['date', 'home', 'away'], how='left')
    df_spread_update = df_spread_update.drop(['spread_favorite_x', 'team_favorite_id_x', 'date'], axis = 1)
    df_spread_update = df_spread_update.rename(columns={"spread_favorite_y" : "spread_favorite", "team_favorite_id_y" : "team_favorite_id"})
    return df_spread_update


In [75]:
df_test2 = update_spreads(df, df_ml)
df_test2.head(10)

,schedule_date,home,score_home,away,score_away,team_favorite_id,spread_favorite
0,2009-09-13,ARI,16.0,SF,20.0,ARI,-5.00
1,2009-09-13,ATL,19.0,MIA,7.0,ATL,-4.00
2,2009-09-13,BAL,38.0,KC,24.0,BAL,-13.00
3,2009-09-13,CAR,10.0,PHI,38.0,PHI,-2.50
4,2009-09-13,CIN,7.0,DEN,12.0,CIN,-4.75
5,2009-09-13,CLE,20.0,MIN,34.0,MIN,-4.00
6,2009-09-13,GB,21.0,CHI,15.0,NaN,NaN
7,2009-09-13,HOU,7.0,NYJ,24.0,HOU,-4.50
8,2009-09-13,IND,14.0,JAX,12.0,NaN,NaN
9,2009-09-13,NO,45.0,DET,27.0,NO,-14.00
